In [ ]:
# Packages Used
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import itertools
import PySimpleGUI as sg
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk

# Data Import
# CSV file with one column for time data and another for the current.
#columns = ["time(s)", "current(pA)"]
file_name = '<file name>' 
data = pd.read_csv(file_name + ".txt", sep=",") #, names = columns)

In [ ]:
# Embedding the Matplotlib Toolbar Application
# ------------------------------- This is to include a matplotlib figure in a Tkinter canvas
_VARS = {'window': False,
         'fig_agg': False,
         'pltFig': False,
         'controls_cv': False}

#Defining the Tkinter canvas and space on the canvas for the i-t trace and toolbar
def draw_figure(canvas, fig, canvas_toolbar):
    if canvas_toolbar.children:
        for child in canvas_toolbar.winfo_children():
            child.destroy()
    if canvas.children:
        for child in canvas.winfo_children():
            child.destroy()
    figure_canvas_agg = FigureCanvasTkAgg(fig, master=canvas)
    figure_canvas_agg.draw()
    toolbar = Toolbar(figure_canvas_agg, canvas_toolbar)
    toolbar.update()
    figure_canvas_agg.get_tk_widget().pack(side='right', fill='both', expand=1)
    return figure_canvas_agg    

#Defining space on the canvas for the histograms
def draw_histogram(histcanvas, hist):
    if histcanvas.children:
        for child in histcanvas.winfo_children():
            child.destroy()
    figure_canvas_agg_hist = FigureCanvasTkAgg(hist, master=histcanvas)
    figure_canvas_agg_hist.draw()
    figure_canvas_agg_hist.get_tk_widget().pack(side='right', fill='both', expand=1)
    return figure_canvas_agg_hist

#Defining the toolbar
class Toolbar(NavigationToolbar2Tk):
    def __init__(self, *args, **kwargs):
        super(Toolbar, self).__init__(*args, **kwargs)

def save_identification():  
    identification_df.to_csv(file_name + "_identification.txt")

def save_deltaiHistogram(): 
    histogram_results_df.to_csv(file_name + "_delta_i_Histogram.txt")

# PySimpleGUI
# ------------------------------- PySimpleGUI CODE
#initial values for N, Mrange, and %T
val1=10		# N number of data points
val2=0.000035 	# Mrange
val3=1			# T, percent tolerance to combine

#initial bin sizes for histograms
hist1 = 30
hist2 = 50
hist3 = 30

#Creating buttons, sliders, and defining the size of the i-t trace figure
layout = [
    [sg.B('Exit', button_color= ('white', 'firebrick3')), sg.Button('Save Identification', button_color= ('white', 'green')), sg.Button('Save Δi Histogram', button_color= ('white', 'green'))],
    [sg.Button('Update'), sg.Button('Reset Axis')],
    [sg.Canvas(key='controls_cv')],
    [sg.T('Number of Data Points:   ', text_color = 'black', background_color = 'white'),
        sg.Slider(range=(1, 100), orientation = 'h', size =(25,15), default_value=val1, background_color = 'white', text_color = 'black', key="slider1", enable_events=True),
        sg.T('      Delta i Bin Size:           ', text_color = 'black', background_color = 'white'),
    sg.Slider(range=(1, 100), orientation = 'h', size =(25,15), default_value=hist1, resolution=1, background_color = 'white', text_color = 'black', key="slider4", enable_events=True)],
    [sg.T('Slope Range (10^-5):       ', text_color = 'black', background_color = 'white'),
        sg.Slider(range=(0.000001, 0.0003), orientation = 'h', size =(25,15), default_value=val2, resolution=0.000001, background_color = 'white', text_color = 'black', key="slider2", enable_events=True),
        sg.T('      Delta t Bin Size:          ', text_color = 'black', background_color = 'white'),
        sg.Slider(range=(1, 100), orientation = 'h', size =(25,15), default_value=hist2, resolution=1, background_color = 'white', text_color = 'black', key="slider5", enable_events=True)],
    [sg.T('% Tolerance to Combine:', text_color = 'black', background_color = 'white'),
        sg.Slider(range=(0.01, 10), orientation = 'h', size =(25,15), default_value=val3, resolution=0.01, background_color = 'white', text_color = 'black', key="slider3", enable_events=True),
        sg.T('      Current Avg Bin Size:   ', text_color = 'black', background_color = 'white'),
        sg.Slider(range=(1, 100), orientation = 'h', size =(25,15), default_value=hist3, resolution=1, background_color = 'white', text_color = 'black', key="slider6", enable_events=True)],
    [sg.Canvas(key='figCanvas',
               size=(600*2.5, 650),
               background_color='#ffffff')]
]

#Defining the size and location of the Tkinter canvas
_VARS['window'] = sg.Window('Step Fit Analysis',
                            layout,
                            finalize=True,
                            size=(1600,950),
                            resizable=True,
                            location=(20, 20),
                            background_color= '#ffffff') #DAE0E6')

#Defining time and current data
def itData():
    xData = data['time(s)']
    yData = data['current(pA)']
    
    return(xData, yData)

# Step Identification Script
#---------------------------------Start of the Step Identification Script

#Defining the step identification function
def StepIdentification():
# Fit Parameters
    #How many data points per section, N.
    number = val1

    #Slope range parameters, Mrange.
    pcutoff= val2
    ncutoff = -val2

    #% tolerance for combining averages, T.
    percent = val3/100
# Data Fitting
############################ Assessing length of data file and how many data points per section.
    a = len(data)
    #print("a =", a, "(number of rows)")
    b=int(a/number)
    #print("b =", b, "(number of rows/n)")

    ############################ Splitting current and time into sections. Separate files for both.
    current_list = data['current(pA)'].tolist() 
    time_list = data['time(s)'].tolist()
    split_time_array=np.array_split(time_list, b)
    split_current_array=np.array_split(current_list, b)

    ############################ Calculating slopes of each section.
    slopes_list = []
    i=0
    while split_current_array[i].any() < len(split_current_array) and split_time_array[i].any() < len(split_time_array):
        y = np.array(split_current_array[i])
        x = np.array(split_time_array[i]).reshape((-1, 1))
        linear=LinearRegression()
        linear.fit(x, y)

        c = linear.coef_
        d = c.tolist()
        slopes_list.append(d)

        i = i+1
        if(i==len(split_current_array)) or (i==len(split_time_array)):
            break

    ############################ Slopes in data frame - not used, just for reference if needed
    slopes_df = pd.DataFrame(slopes_list, columns = ["slopes"])
    #print(slopes_df)

    ############################ Separating sections based on slope. Only taking sections with slopes between parameters.
    flat_current_nestedlist = []
    flat_time_nestedlist = []
    i=0
    while split_current_array[i].any() < len(split_current_array) and split_time_array[i].any() < len(split_time_array):
        f = slopes_list[i]
        g = split_current_array[i]
        h = split_time_array[i]
        if [ncutoff] <= f <= [pcutoff]:
            #print(data_split_current[i])
            flat_current_nestedlist.append(g)
            flat_time_nestedlist.append(h)

        i = i+1
        if(i==len(split_current_array)) or (i==len(split_time_array)):
            break

    ############################ Taking first and last time entry from each time section.
    flat_time_sections_i = []
    i=0
    for i in range(len(flat_time_nestedlist)):
        p = flat_time_nestedlist[i][0],flat_time_nestedlist[i][-1]
        flat_time_sections_i.append(p)

        i = i+1
        if(i==len(split_current_array)) or (i==len(split_time_array)):
            break

    flat_time_sections = list(itertools.chain.from_iterable(flat_time_sections_i))
    #print(len(flat_time_sections))

    ############################ Setting up to duplicate average currents of each section. Needed for plotting purposes. 
    duplicates_2 = []
    i=0
    for i in range(len(flat_time_sections)):
        q=2
        duplicates_2.append(q)

        i = i+1
        if(i==len(flat_time_sections)) or (i==len(flat_time_sections)):
            break

    ############################ Averaging each current section.         
    average_flat_currents_i = []
    i=0
    for i in range(len(flat_current_nestedlist)):
        o = np.mean(flat_current_nestedlist[i])
        average_flat_currents_i.append(o)

        i = i+1
        if(i==len(split_current_array)) or (i==len(split_time_array)):
            break

    ############################ Duplicating average currents for each section. Needed for plotting purposes. 
    def repeat_it(lst, numbers):
        return itertools.chain.from_iterable(itertools.repeat(i, j) for i, j in zip(lst, numbers))
    average_flat_currents = list(repeat_it(average_flat_currents_i, duplicates_2))

    ############################ Combining current and time selections for plotting.
    flat_data_df = pd.DataFrame(flat_time_sections, columns = ['Time(s)'])
    flat_data_df['Current(pA)'] = average_flat_currents
    #print(flat_data_df)

    #Start of code for grouping similar averages
    ############################ #Converting Current column of data frame to list form
    flat_current_grouping = flat_data_df['Current(pA)'].tolist()
    #print(len(flat_current_grouping))

    ############################ #Identifying when average is within % tolerance of surrounding averages. '1' and '2' are used to
    # denote true/false in order to group the averages later.

    temp_current_list = []
    i=0
    for i in range(0,len(flat_current_grouping)):
        p = abs(flat_current_grouping[i-1])
        if p-p*percent <= abs(flat_current_grouping[i]) <= p+p*percent:
            q = 1
            temp_current_list.append(q)
            #print(np.mean(average_flat_currents_i[i]+p))

        else:
            r = 2
            temp_current_list.append(r)

        i = i+1
        if(i==len(flat_current_grouping)):
            break
    #print(temp_current_list)

    ############################ #adding column of 1 and 2 values for conditional true/false of the % tolerance of the averages
    flat_data_df['T/F'] = temp_current_list
    #print(flat_data_df)

    ############################ #splitting the current based on 1 and 2 groupings. Every time there is a 2, a new grouping is formed.
    separator = flat_data_df[flat_data_df['T/F']== 2].index.values
    flat_current_split=np.array_split(flat_data_df['Current(pA)'], separator)
    #print(len(flat_current_split))

    ############################ # taking averages of new combined groupings
    split_mean = []
    i=1
    for i in range(1,len(flat_current_split)):
        s = np.mean(flat_current_split[i])
        split_mean.append(s)

        i = i+1
        if(i==len(flat_current_split)):
            break
    #print(len(split_mean))

    ############################ #splitting the time based on 1 and 2 groupings. Every time there is a 2, a new grouping is formed.
    flat_time_split=np.array_split(flat_data_df['Time(s)'], separator)
    #print(len(flat_time_split))

    ############################ #collecting the beginning and end times of the new combined groupings
    split_time_i = []
    i=1
    for i in range(1,len(flat_time_split)):
        p = flat_time_split[i][::len(flat_time_split[i])-1]
        #p = flat_time_split[i][0],flat_time_split[i][-1]
        split_time_i.append(p)

        i = i+1
        if(i==len(flat_time_split)):
            break

    split_time = list(itertools.chain.from_iterable(split_time_i))
    #print(len(split_time))

    ############################ #need to duplicate the averages for plotting. Making a list of 2s to duplicate current averages.
    duplicates_2_2 = []
    i=1
    for i in range(1, len(flat_time_split)):
        if len(flat_time_split[i]) != 1:
            q=2
            duplicates_2_2.append(q)

        i = i+1
        if(i==len(flat_time_split)):
            break

    #print(len(duplicates_2_2))

    ############################ #duplicating current averages
    def repeat_it(lst, numbers):
        return itertools.chain.from_iterable(itertools.repeat(i, j) for i, j in zip(lst, numbers))
    average_flat_currents_2 = list(repeat_it(split_mean, duplicates_2_2))

    ############################ #data frame for plotting. Contains beginning and end times for each combined segment and duplicated
    # averages for those times
    flat_data_df_2 = pd.DataFrame(split_time, columns = ['Time(s)'])
    flat_data_df_2['Current(pA)'] = average_flat_currents_2
    
    #---------------------------------Step Identification Data
    #Step Fit 
    x2Data = flat_data_df_2['Time(s)']
    y2Data =flat_data_df_2['Current(pA)']

# Histogram    
    #---------------------------------Histogram
    
    #Subtracting Averages of 'combined' data frames
    split_mean_df = pd.DataFrame(split_mean, columns = ['Average Currents (pA)'])
    zero = pd.DataFrame([0])
    split_mean_df_shifted = pd.concat([zero, split_mean_df]).reset_index(drop = True)
    deltai = split_mean_df-split_mean_df_shifted

    #Histogram of Delta i
    histogram = deltai['Average Currents (pA)'].dropna()    

# Time Between Val Events
    #---------------------------------Time between valinomycin events

    #Subtracting time between events
    insertiontime_df = flat_data_df_2.iloc[::2]['Time(s)'].reset_index()
    zero = pd.DataFrame([0])
    insertiontime_df_shifted = pd.concat([zero, insertiontime_df]).reset_index(drop = True)
    timebetweeninsertions = insertiontime_df['Time(s)']-insertiontime_df_shifted['Time(s)']
    

# Average Currents
    #---------------------------------Current Averages = split_mean
    
    return (x2Data, y2Data, histogram, timebetweeninsertions, split_mean)
#---------------------------------End of Step Identification Script. Code beyond this point is for GUI plots   

# Plot Window
# original plot limits for i vs t plot
ylimits = (data['current(pA)'].min()*1.1, data['current(pA)'].max()*1.1)
xlimits = (data['time(s)'].min(), data['time(s)'].max() )

# plot layout
_VARS['pltFig'] = plt.figure(constrained_layout=False)
grid = gridspec.GridSpec(ncols=3, nrows=3, figure=_VARS['pltFig'])
ax1 = _VARS['pltFig'].add_subplot(grid[2, 0])
ax2 = _VARS['pltFig'].add_subplot(grid[:2,:2])
ax3 = _VARS['pltFig'].add_subplot(grid[2, 1])
ax4 = _VARS['pltFig'].add_subplot(grid[2, 2])

#Defining the plots with data
def DrawChart():  
    #Defining the output from StepIdentification()  
    dataXY = itData()
    dataX2 = StepIdentification()[0]
    dataY2 = StepIdentification()[1]
    hisdata = StepIdentification()[2]
    timedata = StepIdentification()[3]
    avgcurrent = StepIdentification()[4]
    
    #Making the plots
    ax2.plot(dataXY[0], dataXY[1], label = 'Current-time data',  linewidth=3)
    ax2.plot( dataX2,  dataY2, label = 'Step Identification', linewidth=2)
    n, bins, patches = ax1.hist(x=hisdata, bins= int(hist1), color='#0504aa',
                            alpha=0.7, rwidth=0.85, label = 'Delta i')
    n, bins, patches = ax3.hist(x=timedata, bins=int(hist2), color='#0504aa',
                            alpha=0.7, rwidth=0.85, label = 'Time between events')
    n, bins, patches = ax4.hist(x=avgcurrent, bins=int(hist3), color='#0504aa',
                            alpha=0.7, rwidth=0.85, label = 'Average Currents')
    
    #Labeling the plots
    ax2.set(xlabel ='Time (s)', ylabel = 'Current (pA)')
    ax2.legend()
    ax1.set(xlabel ='Current (pA)', ylabel = 'Counts')
    ax1.legend()
    ax3.set(xlabel = 'Time(s)')
    ax3.legend()
    ax4.set(xlabel = 'Current (pA)')
    ax4.legend()
    
    #x and y limits of the plots
    ax2.set_ylim(ylimits)
    ax2.set_xlim(xlimits)

    #plot
    _VARS['fig_agg'] = draw_figure(
    _VARS['window']['figCanvas'].TKCanvas, _VARS['pltFig'], _VARS['window']['controls_cv'].TKCanvas)

#Updating plots when parameters change
def updateChart():
    #clearing existing plots    
    ax1.cla()
    ax2.cla()
    ax3.cla()
    ax4.cla()
 
    #Defining the output from StepIdentification()   
    dataXY = itData()
    dataX2 = StepIdentification()[0]
    dataY2 = StepIdentification()[1]
    hisdata = StepIdentification()[2]
    timedata = StepIdentification()[3]
    avgcurrent = StepIdentification()[4]
    
    #Making the plots
    ax2.plot(dataXY[0], dataXY[1], label = 'Current-time data',  linewidth=3)
    ax2.plot( dataX2,  dataY2, label = 'Step Identification', linewidth=2)
    n, bins, patches = ax1.hist(x=hisdata, bins= int(hist1), color='#0504aa',
                            alpha=0.7, rwidth=0.85, label = 'Delta i')
    n, bins, patches = ax3.hist(x=timedata, bins=int(hist2), color='#0504aa',
                            alpha=0.7, rwidth=0.85, label = 'Time between events')
    n, bins, patches = ax4.hist(x=avgcurrent, bins=int(hist3), color='#0504aa',
                            alpha=0.7, rwidth=0.85, label = 'Average Currents')
    
    #Labeling the plots
    ax2.set(xlabel ='Time (s)', ylabel = 'Current (pA)')
    ax2.legend()
    ax1.set(xlabel ='Current (pA)', ylabel = 'Counts')
    ax1.legend()
    ax3.set(xlabel = 'Time(s)')
    ax3.legend()
    ax4.set(xlabel = 'Current (pA)')
    ax4.legend()
  
    #x and y limits of the plots  
    ax2.set_ylim(ylimits)
    ax2.set_xlim(xlimits)

    #plot
    _VARS['fig_agg'] = draw_figure(
    _VARS['window']['figCanvas'].TKCanvas, _VARS['pltFig'], _VARS['window']['controls_cv'].TKCanvas) 

#plotting    
DrawChart()

#Connecting buttons and values of the sliders to the analysis
while True:
    event, values = _VARS['window'].read()
    print(event, values)
    if event in (sg.WIN_CLOSED, 'Exit'):  # always,  always give a way out!
        break

    elif event == "Save Identification":
        #Making i-t data frame of the identification
        idealized_time = StepIdentification()[0]
        idealzed_current = StepIdentification()[1]
        identification_df = pd.DataFrame({"Time (s)":idealized_time, "Current (pA)":idealzed_current})
        save_identification()
        
        #Making text file of the identification parameters
        f= open(file_name + "_identification parameters.txt","w+")
        f.write("Number of Data Points, N = " + str(val1) + "\nSlope Range, M_range = ± " + str(val2) + "\nTolerance Range, T = " + str(val3))
        f.close()
        
    elif event == "Save Δi Histogram":
        histogram_results = StepIdentification()[2]
        histogram_results_df = pd.DataFrame({"Current (pA)":histogram_results})
        save_deltaiHistogram()        


        
    if event == 'Update':
        xlimits = ax2.get_xlim()
        ylimits = ax2.get_ylim()
        updateChart()
        #print(xlimits)
        
    if event == 'Reset Axis':
        ylimits = (data['current(pA)'].min()*1.1, data['current(pA)'].max()*1.1)
        xlimits = (data['time(s)'].min(), data['time(s)'].max())
        updateChart()

    #slider 1
    elif event == "slider1":
        val1 = values["slider1"]
        _VARS['window'].Element("slider1").Update(val1) 
        
    #slider 2
    elif event == "slider2":
        val2 = values["slider2"]
        _VARS['window'].Element("slider2").Update(val2)
        
    #slider 3
    elif event == "slider3":
        val3 = values["slider3"]
        _VARS['window'].Element("slider3").Update(val3)
        
    #slider 4
    elif event == "slider4":
        hist1 = values["slider4"]
        _VARS['window'].Element("slider4").Update(hist1) 
        
    #slider 5
    elif event == "slider5":
        hist2 = values["slider5"]
        _VARS['window'].Element("slider5").Update(hist2)            

    #slider 6
    elif event == "slider6":
        hist3 = values["slider6"]
        _VARS['window'].Element("slider6").Update(hist3)          
        
_VARS['window'].close()
